# Competitive model [Competitive mechanism only]

##Changing medium

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba
from utils import findBiomarkers

In [2]:
M1 = cobra.io.load_json_model('Recon2.2plusthreeCompetitive.json')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [16]:
for reaction in M1.reactions:
    if 'EX_'in reaction.id and M1.reactions.get_by_id(reaction.id).upper_bound>0:
        print(reaction)

EX_phpyr_BD: phpyr_BD --> 
EX_34dhphe_BD: 34dhphe_BD --> 
EX_glc_D_BD: glc_D_BD --> 
EX_phe_L_BD: phe_L_BD --> 
EX_tyr_L_BD: tyr_L_BD --> 
EX_his_L_BD: his_L_BD --> 
EX_ile_L_BD: ile_L_BD --> 
EX_leu_L_BD: leu_L_BD --> 
EX_met_L_BD: met_L_BD --> 
EX_thr_L_BD: thr_L_BD --> 
EX_trp_L_BD: trp_L_BD --> 
EX_val_L_BD: val_L_BD --> 
EX_lys_L_BD: lys_L_BD --> 
EX_ala_L_BD: ala_L_BD --> 
EX_arg_L_BD: arg_L_BD --> 
EX_asn_L_BD: asn_L_BD --> 
EX_asp_L_BD: asp_L_BD --> 
EX_cys_L_BD: cys_L_BD --> 
EX_glu_L_BD: glu_L_BD --> 
EX_gln_L_BD: gln_L_BD --> 
EX_gly_BD: gly_BD --> 
EX_pro_L_BD: pro_L_BD --> 
EX_ser_L_BD: ser_L_BD --> 
EX_o2_BD: o2_BD <=> 
EX_nh4_BD: nh4_BD --> 
EX_pi_BD: pi_BD --> 
EX_acac_BD: acac_BD --> 
EX_bhb_BD: bhb_BD --> 
EX_co2_BD: co2_BD --> 
EX_h_BD: h_BD --> 
EX_h2o_BD: h2o_BD --> 
EX_lac_D_BD: lac_D_BD --> 
EX_lac_L_BD: lac_L_BD --> 
EX_so4_BD: so4_BD --> 
EX_urea_BD: urea_BD --> 
EX_dopa_BD: dopa_BD --> 
EX_adrnl_BD: adrnl_BD --> 
EX_srtn_BD: srtn_BD --> 


## Tyrosine is not enough in the diet

In [3]:
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitiveMap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

model=M1.copy()

In [4]:
model1=model.copy()

In [5]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model1.problem.Objective(model1.reactions.biomass_reaction_INB.flux_expression+
                                              model1.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model1.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model1.objective = quadratic_objective

In [6]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model1.reactions.EX_phe_L_e.lower_bound=-0.2
model1.reactions.EX_phe_L_e.upper_bound=-0.2
model1.reactions.EX_tyr_L_e.lower_bound=-0.04

In [7]:
## Other reactions to produce l-dopa is forbidded
model1.reactions.TYRDOPO_INB.upper_bound=0
model1.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model1.reactions.biomass_reaction.lower_bound=0.05
model1.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model1.reactions.biomass_reaction_INB.upper_bound=0.1
# Phenylpyruvate production in the brain and Liver are limited
model1.reactions.get_by_id('PHPYRte_INB').lower_bound=-0.1
model1.reactions.get_by_id('PHPYRte').lower_bound=-0.1
# Phe secretion in the blood is blocked, otherwise there is no competition.
model1.reactions.EX_phe_L_BD.upper_bound=0
# Serotonin production
model1.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No PAH in the brain
model1.reactions.PHETHPTOX2_INB.lower_bound=0
model1.reactions.PHETHPTOX2_INB.upper_bound=0
model1.reactions.r0399_INB.lower_bound=0
model1.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
# model1.reactions.EX_TH_B_INB.upper_bound=1000
model1.reactions.get_by_id('34dhpheTR_TOBD').upper_bound=0 ##No levodopa from liver to blood.
model1.reactions.Ti1_To1.upper_bound=0.198
### No Tyr hydroxylase competition
model1.reactions.Phe_TH_INB.upper_bound=0
model1.reactions.PhePde_INB.upper_bound=0
model1.solver='glpk'
sol = model1.optimize()
sol.objective_value
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

0.2547554000000001

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [8]:
model2=model1.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model2.genes.get_by_id('HGNC:8582').knock_out()
sol = model2.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.18734518083434298

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [9]:
## Only adding tyrosine
model3=model2.copy()
model3.reactions.EX_tyr_L_e.lower_bound=-0.1
sol = model3.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.18734518083433532

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [10]:
## Phe deprivation and Tyr addition
model3.reactions.EX_phe_L_e.upper_bound=-0.04
model3.reactions.EX_phe_L_e.lower_bound=-0.04
sol = model3.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.2547554000000002

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [11]:
#Only removing the Phe
model4=model2.copy()
model4.reactions.EX_phe_L_e.upper_bound=-0.04
model4.reactions.EX_phe_L_e.lower_bound=-0.04
sol = model4.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.21405359999999996

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [12]:
model5=model2.copy()
# Taking more Phe for PKU patient to show protein synthesis in the brain
model5.reactions.EX_phe_L_e.lower_bound=-0.21
model5.reactions.EX_phe_L_e.upper_bound=-0.21
sol = model5.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.12953369253538088

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

## Tyrosine is enough in the diet

In [13]:
model=M1.copy()
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitiveMap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [14]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model.reactions.EX_phe_L_e.lower_bound=-0.2
model.reactions.EX_phe_L_e.upper_bound=-0.2
model.reactions.EX_tyr_L_e.lower_bound=-0.1

In [15]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [16]:
model1=model.copy()

In [17]:
## Other reactions to produce l-dopa is forbidded
model1.reactions.TYRDOPO_INB.upper_bound=0
model1.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model1.reactions.biomass_reaction.lower_bound=0.05
model1.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model1.reactions.biomass_reaction_INB.upper_bound=0.1
# Phenylpyruvate production in the brain and Liver are limited
model1.reactions.get_by_id('PHPYRte_INB').lower_bound=0
model1.reactions.get_by_id('PHPYRte').lower_bound=0
# Phe secretion in the blood is blocked, otherwise there is no competition.
model1.reactions.EX_phe_L_BD.upper_bound=0
# Serotonin production
model1.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No PAH in the brain
model1.reactions.PHETHPTOX2_INB.lower_bound=0
model1.reactions.PHETHPTOX2_INB.upper_bound=0
model1.reactions.r0399_INB.lower_bound=0
model1.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
# model1.reactions.EX_TH_B_INB.upper_bound=1000
model1.reactions.get_by_id('34dhpheTR_TOBD').upper_bound=0 ##No levodopa from liver to blood.
model1.reactions.Ti1_To1.upper_bound=0.198
### No Tyr hydroxylase competition
model1.reactions.Phe_TH_INB.upper_bound=0
model1.reactions.PhePde_INB.upper_bound=0
model1.solver='glpk'
sol = model1.optimize()
sol.objective_value
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

0.2547554

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [18]:
model2=model1.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model2.genes.get_by_id('HGNC:8582').knock_out()
# Phenylpyruvate production in the brain and Liver are limited and Phe-activated.
model2.reactions.get_by_id('PHPYRte_INB').lower_bound=-0.1
model2.reactions.get_by_id('PHPYRte').lower_bound=-0.1
sol = model2.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.18734518083433538

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [19]:
## Adding more tyrosine
model3=model2.copy()
model3.reactions.EX_tyr_L_e.lower_bound=-0.2
sol = model3.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.18734518083433543

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [20]:
## Only Phe deprivation
model4=model2.copy()
model4.reactions.EX_phe_L_e.upper_bound=-0.04
model4.reactions.EX_phe_L_e.lower_bound=-0.04
sol = model4.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.2547554000000001

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [21]:
## Phe deprivation and Tyr addition
model3.reactions.EX_phe_L_e.upper_bound=-0.04
model3.reactions.EX_phe_L_e.lower_bound=-0.04
sol = model3.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.2547554000000001

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

# Competitive Model [Toxicity mechanism only]

## Tyrosine is enough for producing 0.15 biomass

In [22]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba
model=M1.copy()

In [23]:
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitiveMap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [24]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [25]:
# Changing the phe and tyr in the medium
model.reactions.EX_phe_L_e.lower_bound=-0.2
model.reactions.EX_phe_L_e.upper_bound=-0.2
model.reactions.EX_tyr_L_e.lower_bound=-0.04

In [26]:
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
model.reactions.biomass_reaction_INB.upper_bound=0.1
# minimal serotonin production
model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
# Phenylpyruvate production in the brain and Liver are limited
model.reactions.get_by_id('PHPYRte_INB').lower_bound=0
model.reactions.get_by_id('PHPYRte').lower_bound=0
# Phe secretion in the blood is blocked, otherwise there is no competition.
model.reactions.EX_phe_L_BD.upper_bound=0
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
# model.reactions.EX_TH_B_INB.upper_bound=1000
model.reactions.get_by_id('34dhpheTR_TOBD').upper_bound=0 ##No levodopa from liver to blood.
### Tyrosine hydroxylase competition
model.reactions.TH_AToB_INB.upper_bound=0.07
model.reactions.PhePde_INB.upper_bound=0.02
#
model.solver='glpk'
sol = model.optimize()
sol.objective_value
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

0.26800409999999997

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [27]:
model1=model.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model1.genes.get_by_id('HGNC:8582').knock_out()
# Phenylpyruvate production in the brain and Liver are activated by high Phe concentration
model1.reactions.get_by_id('PHPYRte_INB').lower_bound=-0.1
model1.reactions.get_by_id('PHPYRte').lower_bound=-0.1
model1.reactions.PhePde_INB.upper_bound=0.02
sol = model1.optimize()
fluxP=sol.fluxes.loc['PheTR_TOB']
print(fluxP)
if fluxP>=0.08:
    model1.reactions.Phe_TH_INB.lower_bound=0.055
sol = model1.optimize()    
sol.objective_value
b = show_map(sol,map_loc)
b

0.10392741791710691


0.2130041

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.03906, '1a_24_…

In [28]:
### Tyr supplementation alone
model2=model1.copy()
model2.reactions.EX_tyr_L_e.lower_bound=-0.1
sol = model2.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.2130041

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [29]:
### Phe deprivation and Tyr supplementation
model3=model2.copy()
model3.reactions.Phe_TH_INB.lower_bound=0
model3.reactions.EX_phe_L_e.upper_bound=-0.04
model3.reactions.EX_phe_L_e.lower_bound=-0.04
sol = model3.optimize()
sol.objective_value
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

0.26800409999999997

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [30]:
## Phe deprivation alone
model5=model1.copy()
model5.reactions.Phe_TH_INB.lower_bound=0
model5.reactions.EX_phe_L_e.upper_bound=-0.04
model5.reactions.EX_phe_L_e.lower_bound=-0.04
sol = model5.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.21405359999999996

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

## Tyrosine is not enough for producing 0.15 biomass

In [31]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba
model=M1.copy()

In [32]:
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitiveMap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [33]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [34]:
# Changing the phe and tyr in the medium
model.reactions.EX_phe_L_e.lower_bound=-0.2
model.reactions.EX_phe_L_e.upper_bound=-0.2
model.reactions.EX_tyr_L_e.lower_bound=-0.02

In [35]:
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
model.reactions.biomass_reaction_INB.upper_bound=0.1
# minimal serotonin production
model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
# Phenylpyruvate production in the brain and Liver are limited
model.reactions.get_by_id('PHPYRte_INB').lower_bound=0
model.reactions.get_by_id('PHPYRte').lower_bound=0
# Phe secretion in the blood is blocked, otherwise there is no competition.
model.reactions.EX_phe_L_BD.upper_bound=0
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
# model.reactions.EX_TH_B_INB.upper_bound=1000
model.reactions.get_by_id('34dhpheTR_TOBD').upper_bound=0 ##No levodopa from liver to blood.
### Tyrosine hydroxylase competition
model.reactions.TH_AToB_INB.upper_bound=0.07
model.reactions.PhePde_INB.upper_bound=0.02
#
model.solver='glpk'
sol = model.optimize()
sol.objective_value
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

0.26800409999999997

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [36]:
model1=model.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model1.genes.get_by_id('HGNC:8582').knock_out()
# Phenylpyruvate production in the brain and Liver are activated by high Phe concentration
model1.reactions.get_by_id('PHPYRte_INB').lower_bound=-0.1
model1.reactions.get_by_id('PHPYRte').lower_bound=-0.1
model1.reactions.PhePde_INB.upper_bound=0.02
sol = model1.optimize()
fluxP=sol.fluxes.loc['PheTR_TOB']
print(fluxP)
if fluxP>=0.08:
    model1.reactions.Phe_TH_INB.lower_bound=0.055
sol = model1.optimize()    
sol.objective_value
b = show_map(sol,map_loc)
b

0.08702650000000005


0.17359165779420038

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [37]:
### Tyr supplementation alone
model2=model1.copy()
model2.reactions.EX_tyr_L_e.lower_bound=-0.1
sol = model2.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.21300409999999997

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [38]:
### Phe deprivation and Tyr supplementation
model3=model2.copy()
model3.reactions.Phe_TH_INB.lower_bound=0
model3.reactions.EX_phe_L_e.upper_bound=-0.04
model3.reactions.EX_phe_L_e.lower_bound=-0.04
sol = model3.optimize()
sol.objective_value
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

0.26800409999999997

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [39]:
## Phe deprivation alone
model5=model1.copy()
model5.reactions.Phe_TH_INB.lower_bound=0
model5.reactions.EX_phe_L_e.upper_bound=-0.04
model5.reactions.EX_phe_L_e.lower_bound=-0.04
sol = model5.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.17359165779420035

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

##Tyrosine is not enough for total biomass synthesis: Toxicity mechanism could not describe the biomass synthesis (protein synthesis) in the brain when we add more tyrosine for PKU diseae.
Tyrosine is enough for total biomass synthesis: Toxicity mechanism could not describe the effective of Phe deprivation alone.
So, at least only toxicity mechanism could not add PKU disease.

# Compating the results from two mechanism or only transport competitive

## Competitive Model [Toxicity mechanism and Competitive mechanism]

###Dopamine is more important than serotonin for objective function. We did not set the minimal serotonin production in the model.

In [40]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba
model=M1.copy()

In [41]:
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitiveMap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [42]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              2*model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [43]:
# Changing the phe and tyr in the medium
model.reactions.EX_phe_L_e.lower_bound=-0.2
model.reactions.EX_phe_L_e.upper_bound=-0.2
model.reactions.EX_tyr_L_e.lower_bound=-0.04

In [45]:
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
model.reactions.biomass_reaction_INB.upper_bound=0.1
# minimal serotonin production
# model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.0848
# Phe secretion in the blood is blocked, otherwise there is no competition.
model.reactions.EX_phe_L_BD.upper_bound=0
# Phenylpyruvate production in the brain and Liver are limited
model.reactions.get_by_id('PHPYRte_INB').lower_bound=0
model.reactions.get_by_id('PHPYRte').lower_bound=0
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
# model.reactions.EX_TH_B_INB.upper_bound=1000
model.reactions.get_by_id('34dhpheTR_TOBD').upper_bound=0 ##No levodopa from liver to blood.
model.reactions.Ti1_To1.upper_bound=0.198
### Tyrosine hydroxylase competition
model.reactions.TH_AToB_INB.upper_bound=0.07
model.reactions.PhePde_INB.upper_bound=0.02
model.solver='glpk'
sol = model.optimize()
sol.objective_value
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

0.3247554

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [46]:
model1=model.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model1.genes.get_by_id('HGNC:8582').knock_out()
model1.reactions.get_by_id('PHPYRte_INB').lower_bound=-0.1
model1.reactions.get_by_id('PHPYRte').lower_bound=-0.1
sol = model1.optimize()
fluxP=sol.fluxes.loc['PheTR_TOB']
print(fluxP)
if fluxP>=0.08:
    model1.reactions.Phe_TH_INB.lower_bound=0.055
sol = model1.optimize()
sol.objective_value
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

0.08702649999999995


0.20867590000000008

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [47]:
model2=model1.copy()
model2.reactions.EX_tyr_L_e.lower_bound=-0.1
sol = model2.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.2086759

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

##Now we found only Tyr supplementation is useless even dopamine is more important in the system.

## Try supplementation for only Transport competitive mechanism

In [48]:
model3=model.copy()
model3.genes.get_by_id('HGNC:8582').knock_out()
model3.reactions.get_by_id('PHPYRte_INB').lower_bound=-0.1
model3.reactions.get_by_id('PHPYRte').lower_bound=-0.1
model3.reactions.Phe_TH_INB.upper_bound=0
sol = model3.optimize()
sol.objective_value
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

0.20972539999999995

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [49]:
model3.reactions.EX_tyr_L_e.lower_bound=-0.1
sol = model3.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.26367589999999996

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…